In [42]:
import requests
import json
import pandas as pd
import time
import numpy as np

# Create all csv's needed for wine_db in postgresgl

In [61]:
# Load in appended csv's from three merchant csv obtained by scraping and json created from Cleanup notebook
#   Continue to do a little cleanup...
fileName    = "allwine.csv"
region_df = pd.read_csv(fileName)
region_df=region_df.replace(np.nan,"na")
region_df = region_df[region_df['price']!='na']
region_df['country']=region_df['country'].replace(',','na')
region_df = region_df.reset_index()
region_df = region_df.rename(columns={'index':'wine_id'})
region_df = region_df.fillna(0)
region_df

,wine_id,name,country,region,state,varietal,price,description,merchant,address
0,0,Angels Landing Cabernet Sauvignon,United States,Napa Valley,California,Cabernet Sauvignon,22.99,"This Cabernet has it all. Ripe dark fruit, str...",Gary's Wayne,"1308 NJ-23, Wayne, NJ 07470, USA"
1,1,Angels Landing Merlot Napa,United States,Napa Valley,California,Merlot,16.99,"A rich mix of black fruit, hints of dark choco...",Gary's Wayne,"1308 NJ-23, Wayne, NJ 07470, USA"
2,2,Antale Veneto Rosso,Italy,Veneto,Veneto,Corvina,11.99,Producing every style of wine and with great s...,Gary's Wayne,"1308 NJ-23, Wayne, NJ 07470, USA"
3,3,Articulate Cabernet Sauvignon Walla Walla,United States,Walla Walla Valley,Washington,Cabernet Sauvignon,14.99,na,Gary's Wayne,"1308 NJ-23, Wayne, NJ 07470, USA"
4,4,Baus Family Cabernet Sauvignon Private Reserve,United States,Alexander Valley,California,Cabernet Sauvignon,16.99,"A full-bodied and rich Cabernet, this wine is ...",Gary's Wayne,"1308 NJ-23, Wayne, NJ 07470, USA"
...,...,...,...,...,...,...,...,...,...,...
6081,6088,Candoni Pinot Grigio Organic,Italy,Pavia,Lombardy,Pinot Gris,11.99,Our Organic Pinot Grigio is made from 100% Org...,Petrocks Liquors,"419 Amwell Road Hillsborough, NJ 08844"
6082,6089,Canei Peach,Italy,na,na,na,7.99,Hand it over to an Italian to make an exciting...,Petrocks Liquors,"419 Amwell Road Hillsborough, NJ 08844"
6083,6090,Kendall Jackson Cabernet Grand Reserve,United States,North Coast,California,Cabernet Sauvignon,21.99,Grand Reserve is a winemakerâs blend of our ...,Petrocks Liquors,"419 Amwell Road Hillsborough, NJ 08844"
6084,6091,Bacchus Cabernet Sauvignon,United States,na,California,Cabernet Sauvignon,9.99,David Gordon has been the wine director at Tri...,Petrocks Liquors,"419 Amwell Road Hillsborough, NJ 08844"


In [70]:
# Create a region id table based on (country,region, and state) and save to region.csv to be loaded in postgresql

test1 = region_df.groupby(['country','region', 'state']).size().reset_index().rename(columns={0:'count'})
test2 = test1.reset_index()
test3 = test2.rename(columns={"index": 'region_Id'})
test3=test3.drop(['count'] , axis='columns')
test3

,region_Id,country,region,state
0,0,Argentina,Cafayate,na
1,1,Argentina,Est,na
2,2,Argentina,Mendoza,Cuyo
3,3,Argentina,Mendoza,Mendoza
4,4,Argentina,Mendoza,na
...,...,...,...,...
790,790,United States,na,Washington
791,791,United States,na,na
792,792,Uruguay,na,na
793,793,na,Carneros,California


In [71]:
test3.to_csv('region.csv', encoding='utf-8', index=False)

In [73]:
# Create and clean a dataframe which will be saved as a csv (wine.csv) to be loaded in postgresql 

new_df = pd.merge(region_df, test3,  how='left', left_on=['country','region','state'], right_on = ['country','region','state'])
new_df = new_df.drop(['country','region', 'state','merchant', 'address' ] , axis='columns')
new_df['region_Id'] = new_df['region_Id'].astype(int)
new_df


,wine_id,name,varietal,price,description,region_Id
0,0,Angels Landing Cabernet Sauvignon,Cabernet Sauvignon,22.99,"This Cabernet has it all. Ripe dark fruit, str...",744
1,1,Angels Landing Merlot Napa,Merlot,16.99,"A rich mix of black fruit, hints of dark choco...",744
2,2,Antale Veneto Rosso,Corvina,11.99,Producing every style of wine and with great s...,481
3,3,Articulate Cabernet Sauvignon Walla Walla,Cabernet Sauvignon,14.99,na,777
4,4,Baus Family Cabernet Sauvignon Private Reserve,Cabernet Sauvignon,16.99,"A full-bodied and rich Cabernet, this wine is ...",711
...,...,...,...,...,...,...
6081,6088,Candoni Pinot Grigio Organic,Pinot Gris,11.99,Our Organic Pinot Grigio is made from 100% Org...,432
6082,6089,Canei Peach,na,7.99,Hand it over to an Italian to make an exciting...,522
6083,6090,Kendall Jackson Cabernet Grand Reserve,Cabernet Sauvignon,21.99,Grand Reserve is a winemakerâs blend of our ...,754
6084,6091,Bacchus Cabernet Sauvignon,Cabernet Sauvignon,9.99,David Gordon has been the wine director at Tri...,782


In [65]:
new_df.to_csv('wine.csv', encoding='utf-8', index=False)

In [ ]:
new_df.sort_values(by='Region_Id')

In [74]:
# Create dataframe for merchants to be saved as a csv (merchant.csv) to be loaded in postresql
merchant_address = region_df.groupby(['merchant','address']).nunique()
merchant_address = merchant_address.drop(['wine_id','name','country', 'state', 'region', 'varietal','price', 'description'], axis='columns')
merch_add_df=merchant_address.reset_index()
merch_add_df

,merchant,address
0,Chatham BottleKing,"41 Watchung Ave, Chatham, NJ, 07928"
1,Gary's Wayne,"1308 NJ-23, Wayne, NJ 07470, USA"
2,Petrocks Liquors,"419 Amwell Road Hillsborough, NJ 08844"


In [76]:
merch_add_df.to_csv('merchant.csv', encoding='utf-8', index=False)

In [68]:
# Create a merchant_wine dataframe to be saved as a csv (merchant_wine.csv) to be loaded in postgresql
merchant_wine_df=region_df[['merchant','wine_id']]
merchant_wine_df.reset_index(drop=True, inplace=True)
merchant_wine_df

,merchant,wine_id
0,Gary's Wayne,0
1,Gary's Wayne,1
2,Gary's Wayne,2
3,Gary's Wayne,3
4,Gary's Wayne,4
...,...,...
6081,Petrocks Liquors,6088
6082,Petrocks Liquors,6089
6083,Petrocks Liquors,6090
6084,Petrocks Liquors,6091


In [69]:
merchant_wine_df.to_csv("merchant_wine.csv",encoding='utf-8', index=False)